# RAG

Following: https://python.langchain.com/v0.2/docs/tutorials/rag/

# Data preparation

## Load

In [3]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

## Split

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [13]:
len(splits), splits[0].page_content[:100], len(splits[0].page_content)

(66,
 'LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author',
 969)

## Embed and store

In [14]:
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings


emb_model = OllamaEmbeddings(model="mxbai-embed-large")
vectorstore = Chroma.from_documents(documents=splits, embedding=emb_model)

## Retrieve and generate

In [16]:
from langchain import hub

retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

Rag-prompt is hosted here: https://smith.langchain.com/hub/rlm/rag-prompt

Currently it looks like this:

>You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.  
Question: {question}  
Context: {context}  
Answer:


In [23]:
from langchain_core.runnables import RunnablePassthrough

example_msg = prompt.invoke({"context": "testi", "question": "the question"}).to_messages()
print(example_msg[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: the question 
Context: testi 
Answer:


In [26]:
chain = {"context": retriever , "question": RunnablePassthrough()} | prompt

augmented_prompt = chain.invoke("What is an agent?")
messages = augmented_prompt.to_messages()


You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: What is an agent? 
Context: [Document(page_content='pytest\ndataclasses', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}), Document(page_content='Conversatin samples:\n[\n  {\n    "role": "system",', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}), Document(page_content='}\n]\nThen after these clarification, the agent moved into the code writing mode with a different system message.\nSystem message:', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}), Document(page_content='FILENAME\nCODE\nYou will start with the “entrypoint” file, then go to the ones that are imported by that file, and so on.\nPlease note that the code should be fully functional. No placehol

In [27]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [33]:
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
)

augmented_prompt = chain.invoke("What is an agent?")
messages = augmented_prompt.to_messages()

print(messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: What is an agent? 
Context: pytest
dataclasses

Conversatin samples:
[
  {
    "role": "system",

}
]
Then after these clarification, the agent moved into the code writing mode with a different system message.
System message:

FILENAME
CODE
You will start with the “entrypoint” file, then go to the ones that are imported by that file, and so on.
Please note that the code should be fully functional. No placeholders.
Follow a language and framework appropriate best practice file naming convention.
Make sure that files contain all imports, types etc. Make sure that code in different files are compatible with each other.
Ensure to implement all code, if you are unsure, write a plausible implementation.
Include module dependency or package manager 

In [1]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama3")

In [35]:
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)

response = chain.invoke("What is an agent?")

In [38]:
print(response.content)

Based on the provided context and samples, I don't have enough information to define what an "agent" is in this specific scenario. However, considering the code writing mode and system message, it seems that we are dealing with a Python project. In that case, an agent might refer to a class or object that represents a entity or behavior within the program.


In [39]:
from langchain_core.output_parsers import StrOutputParser

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = chain.invoke("What is an agent?")

In [41]:
print(response)

Based on the context provided, I'm familiar with pytest and dataclasses. However, I don't have specific knowledge about what an "agent" is in this context.

Please clarify or provide more information so I can better assist you.


In [42]:
print(chain.invoke("What are key components for an agent system?"))

{
"thoughts": {
"text": "Key components for an agent system typically include",
"reasoning": "modules or classes that define the agents' behavior and interactions with the environment.",
"plan": [
"- Define a perception module to gather information from the environment.",
"- Implement an action module to select and execute actions."
],
"criticism": "Ensure these components are well-organized, modular, and reusable.",
"speak": "To create a robust agent system, focus on building solid foundation with clear roles for each component."
},
"command": {
"name": ""
}
}


## Using predefined chains

In [43]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is Task Decomposition?"})
print(response["answer"])

{
"thoughts": {
"text": "Task decomposition refers to the process of breaking down a complex task or problem into smaller, more manageable components. This approach helps to clarify the requirements and identify the key steps necessary to complete the task.",
"reasoning": "It's like building with Legos - start with individual pieces and assemble them to form a larger structure.",
"plan": "- Identify the main objective\n- Break down the task into smaller sub-tasks\n- Determine the order of execution",
"criticism": "Don't forget to review and refine your decomposition process as needed to ensure it remains relevant and effective.",
"speak": "Task decomposition is a valuable technique for tackling complex tasks. By breaking them down into smaller, more manageable pieces, we can make steady progress towards our goals."
},
"command": {
"name": "task_decomposition",
"args": {}
}
}


The retrieved context can be accessed in the response:

In [47]:
response["context"]

[Document(page_content='}\n]\nThen after these clarification, the agent moved into the code writing mode with a different system message.\nSystem message:', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
 Document(page_content='Conversatin samples:\n[\n  {\n    "role": "system",', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
 Document(page_content='pytest\ndataclasses', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
 Document(page_content='You should only respond in JSON format as described below\nResponse Format:\n{\n    "thoughts": {\n        "text": "thought",\n        "reasoning": "reasoning",\n        "plan": "- short bulleted\\n- list that conveys\\n- long-term plan",\n        "criticism": "constructive self-criticism",\n        "speak": "thoughts summary to say to user"\n    },\n    "command": {\n        "name": "command name",\n        "args": {\n            "arg name": "value"\n     